In [1]:
# df.columns.tolist()
# X = df['Sentence'] + ' - ' + df['Aspect Term']
# y = df['polarity']
# print(X[0], y[0])

# ignore_symbols = ['.', ',', '!', '?', ';', ':', '(', ')', '[', ']', '{', '}']

# def clean_text(text):
#     text = text.lower()
#     for symbol in ignore_symbols:
#         text = text.replace(symbol, '')
#     return ' '.join(text.split())

# X = [clean_text(x) for x in X]

# print(X[0])

# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report

# tfidf = TfidfVectorizer(ngram_range=(1, 2), min_df=2, max_features=2000)

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# X_train_tfidf = tfidf.fit_transform(X_train)
# X_val_tfidf = tfidf.transform(X_val)

# clg = LogisticRegression(max_iter=1000, class_weight='balanced', n_jobs=1)

# clg.fit(X_train_tfidf, y_train)

# y_pred = clg.predict(X_val_tfidf)
# print(classification_report(y_val, y_pred))

# error_mask = (y_val != y_pred)

# error_df = pd.DataFrame({
#     "text": X_val,
#     "true_label": y_val,
#     "pred_label": y_pred
# })

# error_df = error_df[error_mask]

# for _, row in error_df.head(5).iterrows():
#     print("TEXT:", row["text"])
#     print("TRUE:", row["true_label"])
#     print("PRED:", row["pred_label"])
#     print("-" * 50)

In [2]:
import pandas as pd
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

c:\Code\Python\Projects\NLP Feedback Analyzer\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('../data/Restaurants_Train_v2.csv')

df.head()

,id,Sentence,Aspect Term,polarity,from,to
0,3121,But the staff was so horrible to us.,staff,negative,8,13
1,2777,"To be completely fair, the only redeeming fact...",food,positive,57,61
2,1634,"The food is uniformly exceptional, with a very...",food,positive,4,8
3,1634,"The food is uniformly exceptional, with a very...",kitchen,positive,55,62
4,1634,"The food is uniformly exceptional, with a very...",menu,neutral,141,145


In [4]:
X = df['Sentence'] + ' [SEP] ' + df['Aspect Term']

label2id = {
    "negative": 0,
    "neutral": 1,
    "positive": 2,
    "conflict": 3
}

y = df["polarity"].map(label2id)

id2label = {v: k for k, v in label2id.items()}

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [6]:
def tokenize(text):
    return tokenizer(text.tolist(), padding=True, truncation=True, max_length=128, return_tensors="pt")

In [7]:
train_enc = tokenize(X_train)
val_enc = tokenize(X_val)

In [8]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        item = {k: v[index] for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[index])
        return item

train_dataset = ReviewDataset(train_enc, y_train)
val_dataset = ReviewDataset(val_enc, y_val)

In [9]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4, id2label=id2label, label2id=label2id)

model.to('cuda')

Loading weights: 100%|██████████| 100/100 [00:00<00:00, 704.00it/s, Materializing param=distilbert.transformer.layer.5.sa_layer_norm.weight]   
DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_projector.bias    | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
pre_classifier.bias     | MISSING    | 
classifier.bias         | MISSING    | 
pre_classifier.weight   | MISSING    | 
classifier.weight       | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSelfAttention(
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [10]:
training_args = TrainingArguments(
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    weight_decay=0.01
)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

Step,Training Loss
500,0.673049


Writing model shards: 100%|██████████| 1/1 [00:00<00:00,  1.77it/s]


TrainOutput(global_step=555, training_loss=0.6504903638685072, metrics={'train_runtime': 129.8227, 'train_samples_per_second': 68.262, 'train_steps_per_second': 4.275, 'total_flos': 210947367363264.0, 'train_loss': 0.6504903638685072, 'epoch': 3.0})

In [12]:
preds = trainer.predict(val_dataset)
y_pred = preds.predictions.argmax(axis=1)

print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.65      0.63      0.64       161
           1       0.51      0.43      0.47       127
           2       0.83      0.92      0.87       433
           3       0.00      0.00      0.00        18

    accuracy                           0.75       739
   macro avg       0.50      0.49      0.49       739
weighted avg       0.72      0.75      0.73       739



c:\Code\Python\Projects\NLP Feedback Analyzer\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Code\Python\Projects\NLP Feedback Analyzer\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Code\Python\Projects\NLP Feedback Analyzer\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifi

In [17]:
trainer.save_model('../model')
tokenizer.save_pretrained('../model')

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards: 100%|██████████| 1/1 [00:00<00:00,  1.79it/s]


('../model\\tokenizer_config.json', '../model\\tokenizer.json')